In [19]:
# https://www.quantamagazine.org/a-triplet-tree-forms-one-of-the-most-beautiful-structures-in-math-20231212/

In [20]:
#Here's a conceptual framework for how we might adapt these ideas:
#Increasing Sequence: Look for periods where stock prices increase consistently over a set number of days,
#similar to the concept of increasing Markov numbers with a constant numerator and increasing denominator.
#Decreasing Sequence: Identify periods where stock prices decrease consistently, akin to the increasing numerator 
#with a constant denominator in Aigner's conjecture.
#Stable Sum: Find periods where the sum of closing prices over a set number of days remains constant,
#reflecting Aigner's idea of a constant sum of numerator and denominato

In [21]:
import yfinance as yf
import pandas as pd

# Retrieve AAPL stock data
def get_aapl_data():
    aapl = yf.Ticker("TH")
    data = aapl.history(period="1y")
    return data['Close']

# Analyze data for anomalies
def find_anomalies(data):
    anomalies = []
    window_size = 5  # Example window size

    # Increasing sequence (similar to increasing denominator)
    for i in range(len(data) - window_size):
        if all(data[j] < data[j + 1] for j in range(i, i + window_size - 1)):
            anomalies.append((data.index[i], data[i]))

    # Decreasing sequence (similar to increasing numerator)
    for i in range(len(data) - window_size):
        if all(data[j] > data[j + 1] for j in range(i, i + window_size - 1)):
            anomalies.append((data.index[i], data[i]))

    # Stable sum (constant sum of values)
    for i in range(len(data) - window_size):
        if abs(sum(data[i:i + window_size]) - sum(data[i + 1:i + window_size + 1])) < 1:  # Threshold for stability
            anomalies.append((data.index[i], data[i]))

    return anomalies

# Get AAPL data
aapl_data = get_aapl_data()

# Find anomalies
anomalies = find_anomalies(aapl_data)

# Print anomalies
for date, price in anomalies:
    print(f"Date: {date}, Anomalous Price: {round(price, 2)}")


Date: 2023-02-09 00:00:00-05:00, Anomalous Price: 14.58
Date: 2023-02-10 00:00:00-05:00, Anomalous Price: 14.64
Date: 2023-05-10 00:00:00-04:00, Anomalous Price: 13.95
Date: 2023-05-11 00:00:00-04:00, Anomalous Price: 14.52
Date: 2023-01-20 00:00:00-05:00, Anomalous Price: 17.93
Date: 2023-01-23 00:00:00-05:00, Anomalous Price: 17.46
Date: 2023-01-24 00:00:00-05:00, Anomalous Price: 17.4
Date: 2023-01-25 00:00:00-05:00, Anomalous Price: 17.35
Date: 2023-02-16 00:00:00-05:00, Anomalous Price: 16.82
Date: 2023-02-17 00:00:00-05:00, Anomalous Price: 16.34
Date: 2023-02-21 00:00:00-05:00, Anomalous Price: 15.9
Date: 2023-03-14 00:00:00-04:00, Anomalous Price: 17.31
Date: 2023-03-21 00:00:00-04:00, Anomalous Price: 17.09
Date: 2023-03-22 00:00:00-04:00, Anomalous Price: 16.61
Date: 2023-03-29 00:00:00-04:00, Anomalous Price: 15.57
Date: 2023-05-22 00:00:00-04:00, Anomalous Price: 15.78
Date: 2023-05-23 00:00:00-04:00, Anomalous Price: 15.19
Date: 2023-05-24 00:00:00-04:00, Anomalous Price: 

C:\Users\DELL\AppData\Local\Temp\ipykernel_21756\4002490006.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if all(data[j] < data[j + 1] for j in range(i, i + window_size - 1)):
C:\Users\DELL\AppData\Local\Temp\ipykernel_21756\4002490006.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  anomalies.append((data.index[i], data[i]))
C:\Users\DELL\AppData\Local\Temp\ipykernel_21756\4002490006.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `s

In [22]:
import yfinance as yf
import pandas as pd

# Retrieve AAPL stock data and find anomalies (use functions from the previous code)

# Initialize variables
initial_portfolio = 100000
portfolio = initial_portfolio
max_drawdown = initial_portfolio * 0.70  # 30% max drawdown
trade_records = []
open_position = False
last_sell_date = None

# Analyze each anomaly for trading
for anomaly_date, buy_price in anomalies:
    if open_position or (last_sell_date and anomaly_date <= last_sell_date):
        continue

    sell_date = None
    sell_price = None
    for date, price in aapl_data[anomaly_date:].items():
        if price <= buy_price * 0.95 or price >= buy_price * 1.10:
            sell_date = date
            sell_price = price
            break

    if sell_date:
        percent_change = (sell_price - buy_price) / buy_price
        trade_profit = percent_change * portfolio
        portfolio += trade_profit

        trade_records.append({
            "Buy Date": anomaly_date,
            "Buy Price": buy_price,
            "Sell Date": sell_date,
            "Sell Price": sell_price,
            "% Change": percent_change * 100,
            "Portfolio Value": portfolio,
            "Open Position": False
        })

        last_sell_date = sell_date
        open_position = False

        if portfolio < max_drawdown:
            print("Trading stopped due to max drawdown limit reached.")
            break
    else:
        open_position = True

# Include any open positions in the record
if open_position:
    trade_records.append({
        "Buy Date": anomaly_date,
        "Buy Price": buy_price,
        "Sell Date": None,
        "Sell Price": None,
        "% Change": None,
        "Portfolio Value": portfolio,
        "Open Position": True
    })

# Convert trade records to DataFrame
trade_df = pd.DataFrame(trade_records)
# Display the DataFrame (Color formatting requires additional setup)
trade_df

,Buy Date,Buy Price,Sell Date,Sell Price,% Change,Portfolio Value,Open Position
0,2023-02-09 00:00:00-05:00,14.580000,2023-02-15 00:00:00-05:00,16.059999,10.150889,110150.888546,False
1,2023-05-10 00:00:00-04:00,13.950000,2023-05-15 00:00:00-04:00,15.360000,10.107526,121284.418163,False
2,2023-05-22 00:00:00-04:00,15.785000,2023-05-25 00:00:00-04:00,14.540000,-7.887234,111718.432215,False
3,2023-06-14 00:00:00-04:00,15.070000,2023-06-16 00:00:00-04:00,14.120000,-6.303914,104675.798410,False
4,2023-07-10 00:00:00-04:00,13.550000,2023-07-14 00:00:00-04:00,12.720000,-6.125461,98263.923615,False
5,2023-10-10 00:00:00-04:00,16.360001,2023-10-12 00:00:00-04:00,15.400000,-5.867977,92497.819666,False
6,2023-11-01 00:00:00-04:00,14.190000,2023-11-06 00:00:00-05:00,12.700000,-10.500351,82785.223879,False
7,2023-11-17 00:00:00-05:00,11.530000,2023-11-27 00:00:00-05:00,10.920000,-5.290544,78405.435561,False
8,2023-11-28 00:00:00-05:00,10.870000,2023-12-13 00:00:00-05:00,9.910000,-8.831647,71480.944124,False
9,2023-12-15 00:00:00-05:00,9.040000,NaT,NaN,NaN,71480.944124,True
